In [2]:
!pip install plotly pandas numpy

import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import pytz
data = pd.read_csv("/content/Google-Playstore.csv")
data.head ()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


Task 1: Scatter Plot – Revenue vs Installs (Paid Apps Only)

In [12]:
# Clean the data
data['Installs'] = data['Installs']
# Check if 'Price' column is of type object (string) before applying str.replace
if data['Price'].dtype == object:
    data['Price'] = data['Price'].str.replace('$', '', regex=True).astype(float)
else:
    # If 'Price' is already numeric, handle potential '$' signs differently
    # (e.g., if '$' is present, treat as string and apply str.replace, otherwise, leave as is)
    data['Price'] = pd.to_numeric(data['Price'].astype(str).str.replace('$', '', regex=True), errors='coerce')

data['Revenue'] = data['Price'] * data['Installs']

# Filter paid apps
paid_apps = data[(data['Price'] == 'Paid').notna() & data['Installs'].notna() & data['Category'].notna()]

# Create the scatter plot
fig = px.scatter(
    paid_apps,
    x='Installs',
    y='Price',
    color='Category',
    trendline='ols',
    title='Price vs Installs for Paid Apps'
)

# Display the plot
fig.show()

Output hidden; open in https://colab.research.google.com to view.

Task 2: Grouped Bar Chart (3PM–5PM IST)

In [14]:
def is_between_3_5_pm_ist():
    tz = pytz.timezone('Asia/Kolkata')
    now = datetime.now(tz)
    return now.hour == 15 or now.hour == 16

# Prepare columns
data['Size_MB'] = pd.to_numeric(data['Size'].str.replace('M', '').replace('Varies with device', np.nan), errors='coerce')
data['Last Updated'] = pd.to_datetime(data['Last Updated'], errors='coerce')

jan_apps = data[
    (data['Rating'] >= 4.0) &
    (data['Size_MB'] > 10) &
    (data['Last Updated'].dt.month == 1)
]

# Top 10 categories by installs
top_cats = jan_apps.groupby('Category')['Installs'].sum().nlargest(10).index
filtered = jan_apps[jan_apps['Category'].isin(top_cats)]

# Group by category
summary = filtered.groupby('Category').agg({'Rating': 'mean', 'Content Rating': 'sum'}).reset_index()

# Plot only between 3–5 PM IST
if is_between_3_5_pm_ist():
    fig = px.bar(
        summary,
        x='Category',
        y=['Rating', 'Content Rating'],
        barmode='group',
        title='Avg Rating & Total Content Rating (Top 10 Categories, Updated in Jan)'
    )
    fig.show()
else:
    print("⏳ This chart is only visible between 3PM–5PM IST.")


⏳ This chart is only visible between 3PM–5PM IST.


Task 3: Bubble Chart with Complex Filtering (5PM–7PM IST)

In [18]:
def is_between_5_7_pm_ist():
    tz = pytz.timezone('Asia/Kolkata')
    now = datetime.now(tz)
    return now.hour == 17 or now.hour == 18

# Filter criteria
categories = ['GAME', 'BEAUTY', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'ENTERTAINMENT', 'SOCIAL', 'EVENT']
data['App Name'] = data['App Name'].astype(str)

filtered_bubble = data[
    (data['Category'].isin(categories)) &
    (data['Rating'] > 3.5) &
    (~data['App Name'].str.contains('S', case=False)) &
    (data['Installs'] > 50000)
]

# Clean size
filtered_bubble['Size_MB'] = pd.to_numeric(filtered_bubble['Size'].str.replace('M', '').replace('Varies with device', np.nan), errors='coerce')

# Translations
translations = {
    'BEAUTY': 'सौंदर्य',       # Hindi
    'BUSINESS': 'வணிகம்',     # Tamil
    'DATING': 'Partnersuche'   # German
}
filtered_bubble['Translated_Category'] = filtered_bubble['Category'].apply(lambda x: translations.get(x, x))

# Plot only between 5–7 PM IST
if is_between_5_7_pm_ist():
    fig = px.scatter(
        filtered_bubble,
        x='Size_MB',
        y='Rating',
        size='Installs',
        color='Translated_Category',
        hover_name='App Name',
        title='Size vs Rating (Bubble = Installs)'
    )

    # Highlight Game category in pink
    fig.for_each_trace(
        lambda t: t.update(marker=dict(color='pink')) if 'GAME' in t.name else None
    )

    fig.show()
else:
    print("⏳ This chart is only visible between 5PM–7PM IST.")


⏳ This chart is only visible between 5PM–7PM IST.
